# Artifact 6

## Using OR Tools on Textbook question 

This artifact was used for me to practice using OR Tools. For my group project the use of OR tools has come up several times and I am yet to use this tool on my own. Hence I altered multiple questions I have found to create my own problem and then solve it using OR Tools.

### Question I cam up with
Nurses typically work three days a week and often it is 3 days back to back to back and then 4 days off. 

For the sake of this question lets assume a nurse works three consecutive days within a week. The problem we want to solve is the amount of nurses BC Children's Hospital has to hire in order to have enough workers for every shift. 

Lets consider a simple problem first and expand on that (numbers are randomly selected).


| Day of The Week | Number of Employees Needed | 
|:--------:|:--------:|
|  Monday   |  160   |  
|  Tuesday  |  120   | 
|  Wednesday   |  180  | 
|  Thursday   |  130 | 
|  Friday   |  150  | 
|  Saturday   |  90 | 
|  Sunday   |  70 | 

What we know:

- We want to minimize the total number of employee's to hire
- Each nurse works three consecutive days and then has 4 days off

Lets also add one more constraint:
- Lets says that at least half of the workers get weekends off as well

## Setting up the Problem

Instead of having the decision variables be the day the worker is working, it is easier to think of the decision variable and the start date of a shift. 

In other words we'd have:

$X_1$ = Number of nurses that start their work on Monday <br>
$X_2$ = Number of nurses that start their work on Tuesday <br>
$X_3$ = Number of nurses that start their work on Wednesday <br>
$X_4$ = Number of nurses that start their work on Thursday <br>
$X_5$ = Number of nurses that start their work on Friday <br>
$X_6$ = Number of nurses that start their work on Saturday <br>
$X_7$ = Number of nurses that start their work on Sunday <br>

**Why do we set it up like this?**

If we set it up like this, every decision variable is associated to workers and we are not counting that same worker in another decision variable.

For example if we just made the decision variables the day of the week then a worker how works Monday, Tuesday, Wednesday would be included in all of $ X_1, X_2, X_3 $.

**Objective Function**

We can now set our objective function to be:

$
minimize \sum_{i=1}^{7} X_i
$

Such that: <br>

$X_1 + X_6 + X_7 >= 160$


## Constraints

The constraints for the optimization problem are as follows:

1. $ X_1 + X_6 + X_7 \geq 160 $
2. $ X_1 + X_2 + X_7 \geq 120 $
3. $ X_1 + X_2 + X_3 \geq 180 $
4. $ X_2 + X_3 + X_4 \geq 130 $
5. $ X_3 + X_4 + X_5 \geq 150 $
6. $ X_4 + X_5 + X_6 \geq 90 $
7. $ X_5 + X_6 + X_7 \geq 70 $
8. $ X1 + X2 + X3 - X4 - X5 - X6 - X7 \geq 0 $

Each $ X_i $ where $ i = 1, 2, 3, 4, 5, 6, 7 $ must be non-negative $( X_i \geq 0 $).


In [4]:
# pip install ortools

In [5]:
from ortools.linear_solver import pywraplp

def solve_optimization_problem():
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('GLOP')

    # Variables
    X1 = solver.NumVar(0, solver.infinity(), 'X1')
    X2 = solver.NumVar(0, solver.infinity(), 'X2')
    X3 = solver.NumVar(0, solver.infinity(), 'X3')
    X4 = solver.NumVar(0, solver.infinity(), 'X4')
    X5 = solver.NumVar(0, solver.infinity(), 'X5')
    X6 = solver.NumVar(0, solver.infinity(), 'X6')
    X7 = solver.NumVar(0, solver.infinity(), 'X7')

    # Objective function: Minimize X1 + X2 + X3 + X4 + X5 + X6 + X7
    solver.Minimize(X1 + X2 + X3 + X4 + X5 + X6 + X7)

    # Constraints
    solver.Add(X1 + X6 + X7 >= 160)
    solver.Add(X1 + X2 + X7 >= 120)
    solver.Add(X1 + X2 + X3 >= 180)
    solver.Add(X2 + X3 + X4 >= 130)
    solver.Add(X3 + X4 + X5 >= 150)
    solver.Add(X4 + X5 + X6 >= 90)
    solver.Add(X5 + X6 + X7 >= 70)
    solver.Add(X1 + X2 + X3 - X4 - X5 - X6 - X7 >= 0)

    # Solve the system.
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('X1 =', X1.solution_value())
        print('X2 =', X2.solution_value())
        print('X3 =', X3.solution_value())
        print('X4 =', X4.solution_value())
        print('X5 =', X5.solution_value())
        print('X6 =', X6.solution_value())
        print('X7 =', X7.solution_value())
        print('Objective value =', solver.Objective().Value())
    else:
        print('The problem does not have an optimal solution.')

if __name__ == '__main__':
    solve_optimization_problem()

Solution:
X1 = 89.99999999999999
X2 = 0.0
X3 = 90.00000000000003
X4 = 59.99999999999997
X5 = 0.0
X6 = 40.0
X7 = 30.000000000000004
Objective value = 310.0


### Why are there decimals:

The decimal points you see in the solution values for X3 and X4 (like 
9.000000000000002 and 
5.999999999999997) typically arise from the floating-point arithmetic used by computers to solve linear programming problems, including those handled by OR-Tools. <br>

Computers use binary floating-point arithmetic to handle real numbers, which can only approximate most decimal fractions. This system has inherent precision limitations, so numbers that we expect to be represented exactly as simple decimals often end up with small errors in their least significant digits.

## Lets add more to this

What if we wanted to consider the following

1) Different levels of the nurse (senior/junior)

**How does our problem switch**

Lets say that a senior worker cost double a junior worker
Lets say we need at least 20% of the workers on each day to be senior workers

Variables:

$x_{i, d}$ = Number of workers on starting on day i and of experience type d <br>



In [17]:
def solve_optimization_problem():
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('GLOP')

    # Variables for junior and senior nurses for each day
    X1j = solver.NumVar(0, solver.infinity(), 'X1j')  
    X2j = solver.NumVar(0, solver.infinity(), 'X2j')
    X3j = solver.NumVar(0, solver.infinity(), 'X3j')
    X4j = solver.NumVar(0, solver.infinity(), 'X4j')
    X5j = solver.NumVar(0, solver.infinity(), 'X5j')
    X6j = solver.NumVar(0, solver.infinity(), 'X6j')
    X7j = solver.NumVar(0, solver.infinity(), 'X7j')

    X1s = solver.NumVar(0, solver.infinity(), 'X1s')
    X2s = solver.NumVar(0, solver.infinity(), 'X2s')
    X3s = solver.NumVar(0, solver.infinity(), 'X3s')
    X4s = solver.NumVar(0, solver.infinity(), 'X4s')
    X5s = solver.NumVar(0, solver.infinity(), 'X5s')
    X6s = solver.NumVar(0, solver.infinity(), 'X6s')
    X7s = solver.NumVar(0, solver.infinity(), 'X7s')

    # Objective function: Minimize total cost of nurses
    solver.Minimize(
        X1j + X2j + X3j + X4j + X5j + X6j + X7j +
        2 * (X1s + X2s + X3s + X4s + X5s + X6s + X7s)
    )

    # Constraints
    # Sum of junior and senior nurses must meet the staffing requirements
    solver.Add(X1j + X1s + X6j + X6s + X7j + X7s >= 160)
    solver.Add(X1j + X1s + X2j + X2s + X7j + X7s >= 120)
    solver.Add(X1j + X1s + X2j + X2s + X3j + X3s >= 180)
    solver.Add(X2j + X2s + X3j + X3s + X4j + X4s >= 130)
    solver.Add(X3j + X3s + X4j + X4s + X5j + X5s >= 150)
    solver.Add(X4j + X4s + X5j + X5s + X6j + X6s >= 90)
    solver.Add(X5j + X5s + X6j + X6s + X7j + X7s >= 70)

    # Additional constraint: At least 20% of the workers on days must be senior nurses
    solver.Add(X1s >= 0.2 * (X1j + X1s))
    solver.Add(X2s >= 0.2 * (X2j + X2s))
    solver.Add(X3s >= 0.2 * (X3j + X3s))
    solver.Add(X4s >= 0.2 * (X4j + X4s))
    solver.Add(X5s >= 0.2 * (X5j + X5s))
    solver.Add(X6s >= 0.2 * (X6j + X6s))
    solver.Add(X7s >= 0.2 * (X7j + X7s))

    # Solve the system.
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('X1j =', X1j.solution_value(), 'X1s =', X1s.solution_value())
        print('X2j =', X2j.solution_value(), 'X2s =', X2s.solution_value())
        print('X3j =', X3j.solution_value(), 'X3s =', X3s.solution_value())
        print('X4j =', X4j.solution_value(), 'X4s =', X4s.solution_value())
        print('X5j =', X5j.solution_value(), 'X5s =', X5s.solution_value())
        print('X6j =', X6j.solution_value(), 'X6s =', X6s.solution_value())
        print('X7j =', X7j.solution_value(), 'X7s =', X7s.solution_value())
        print('Objective value =', solver.Objective().Value())
    else:
        print('The problem does not have an optimal solution.')

if __name__ == '__main__':
    solve_optimization_problem()


Solution:
X1j = 87.99999999999999 X1s = 21.999999999999996
X2j = 0.0 X2s = 0.0
X3j = 56.00000000000002 X3s = 14.000000000000005
X4j = 47.99999999999998 X4s = 11.999999999999995
X5j = 15.999999999999995 X5s = 3.9999999999999987
X6j = 32.0 X6s = 8.0
X7j = 8.000000000000005 X7s = 2.0000000000000013
Objective value = 372.0


## Improvements

I believe I familiarized myself enough with OR Tools but the problem I solved doesn't involve a lot of data. Coming up with the problem statement was the hardest part of this artifact. I would like to try the same setup with more data.

Additionally I want to add more constraints to make this problem more realistic. For example instead of just shifts per day include night shifts and day shifts and add an upper bound to the number of night shifts a worker can have per week.